# Protein inference from a list of peptides

This simple example shows how to perform protein inference and protein group-level FDR estimation on a list of peptides with corresponding score and associated proteins. 

The example here is based on the Sage results from in the `data/sage_example` from a dataset consistsing of 9 RAW files with different pools of PrESTs spiked into an E. coli background.
The dataset is described in this publication: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6474350/

In [1]:
import pandas as pd
import re

from picked_group_fdr import fdr
from picked_group_fdr import picked_group_fdr

In [2]:
filename = 'peptide_info_list.tsv'
delimiter = '\t'
peptide_col = 0
protein_col = 1
score_col = 2

1. Read in the file and remove peptide modifications

In [3]:
df = pd.read_csv(filename, sep=delimiter)

def remove_modifications(modified_peptide: str):
    """
    Removes modifications in [] or (). Copied from picked_group_fdr.helpers for convenience.

    Warning! Dirty hack: last removal of ) is needed because some MQ versions have nested parentheses for modifications
    """
    return re.sub(r"\[[^]]*\]", "", re.sub(r"\([^)]*\)", "", modified_peptide)).replace(")", "")

df['peptide'] = df['peptide'].apply(remove_modifications)
df['proteins'] = df['proteins'].str.split(";")
df

,peptide,proteins,score
0,HSQTTAIPSCPEGTVPLYSGFSFLFVQGNQRA,[HPRR4000324_poolB],3.686401
1,LPFPEPYILVYANDAAISEPESVVSSLQGHR,[HPRR2310052_poolA],3.677294
2,HSQTTAIPSCPEGTVPLYSGFSFLFVQGNQRA,[HPRR4000324_poolB],3.667281
3,HSQTTAIPSCPEGTVPLYSGFSFLFVQGNQRA,[HPRR4000324_poolB],3.662397
4,QVALQTFGNQTTIIPAGGAGYK,[HPRR1950303_poolA],3.658609
...,...,...,...
43404,GDSGKYYFQVER,[HPRR4290965_entrapment],0.662155
43405,DDVVFFIYLYQR,[HPRR4320452_entrapment],0.643730
43406,VTVEKELYTWVNMQGGTK,[HPRR4320480_entrapment],0.634876
43407,KGPSFADMEVLYWTHVK,[HPRR4320134_entrapment],0.620052


2. Sort the PSMs by descending score and keep only the highest scoring PSM per peptide

In [4]:
df = df.sort_values(by='score', ascending=False)
df = df.drop_duplicates(subset='peptide', keep='first')
df

,peptide,proteins,score
0,HSQTTAIPSCPEGTVPLYSGFSFLFVQGNQRA,[HPRR4000324_poolB],3.686401
1,LPFPEPYILVYANDAAISEPESVVSSLQGHR,[HPRR2310052_poolA],3.677294
4,QVALQTFGNQTTIIPAGGAGYK,[HPRR1950303_poolA],3.658609
5,STSTGSAASSGSNSSASSEQGLLGR,"[HPRR2700178_poolA, HPRR3760765_poolB]",3.649064
8,KSNTIQSDGISDSEKCSPTVSQGK,[HPRR2210165_poolA],3.622915
...,...,...,...
43402,VQVERSKPNGCR,[rev_HPRR4340087_entrapment],0.669512
43403,DQLQVLVSKQNSIIEELEK,[HPRR3720421_poolB],0.663226
43404,GDSGKYYFQVER,[HPRR4290965_entrapment],0.662155
43405,DDVVFFIYLYQR,[HPRR4320452_entrapment],0.643730


3. Estimate posterior error probabilities from the peptide level scores

In [5]:
peptide_scores = list(df[['score', 'proteins']].to_records(index=False))
df['posterior_error_probability'], _ = fdr.calculate_peptide_fdrs(peptide_scores, keep_decoys=True)

2025-03-26 22:19:57,357 - INFO - #Peptides: Target = 5224; Decoys = 3368
2025-03-26 22:19:57,358 - INFO -     Entrapments = 2324; Targets-Entrapments = 2900
2025-03-26 22:19:57,360 - INFO - #Target peptides at 1% decoy FDR: 2182
2025-03-26 22:19:57,361 - INFO - #Target peptides at 1% entrapment FDR: 2093
2025-03-26 22:19:57,361 - INFO - Decoy FDR at 1% entrapment FDR: 0.0033
2025-03-26 22:19:57,362 - INFO - Entrapment FDR at 1% decoy FDR: 0.0083


4. Convert the peptide list into a dictionary of `peptide_sequence => (peptide_posterior_error_prob, [protein1, ...])`

In [6]:
peptide_info_dict = df.set_index('peptide')[['posterior_error_probability', 'proteins']].to_dict('split')
peptide_info_dict = dict(zip(peptide_info_dict['index'], peptide_info_dict['data']))
list(peptide_info_dict.items())[:10]

[('HSQTTAIPSCPEGTVPLYSGFSFLFVQGNQRA',
  [0.000496031746031746, ['HPRR4000324_poolB']]),
 ('LPFPEPYILVYANDAAISEPESVVSSLQGHR',
  [0.000496031746031746, ['HPRR2310052_poolA']]),
 ('QVALQTFGNQTTIIPAGGAGYK', [0.000496031746031746, ['HPRR1950303_poolA']]),
 ('STSTGSAASSGSNSSASSEQGLLGR',
  [0.000496031746031746, ['HPRR2700178_poolA', 'HPRR3760765_poolB']]),
 ('KSNTIQSDGISDSEKCSPTVSQGK', [0.000496031746031746, ['HPRR2210165_poolA']]),
 ('YNPEGKQDILNQTIDFEGFK', [0.000496031746031746, ['HPRR3890843_poolB']]),
 ('GGGAGFISGLSYLELDNPAGNK', [0.000496031746031746, ['HPRR4180150_poolB']]),
 ('ASAQQENSSTCIGSAIKSESGNSAR', [0.000496031746031746, ['HPRR3000481_poolA']]),
 ('NLAVAMCSRYDSIPVSTSLLGDTSDTTSTGLAQR',
  [0.000496031746031746, ['HPRR2650003_poolA']]),
 ('RWNQTCELLPVSQASWACNLILGAPDSQK',
  [0.000496031746031746, ['HPRR4110155_poolB']])]

5. Run `picked_group_fdr.get_protein_group_results` on this dictionary.

In [7]:
results = picked_group_fdr.get_protein_group_results(peptide_info_dict)

2025-03-26 22:19:57,520 - INFO - Grouping proteins (subset strategy):
2025-03-26 22:19:57,540 - INFO - Generating protein groups from observed peptides
2025-03-26 22:19:57,564 - INFO - Assigning peptides to protein groups
2025-03-26 22:19:57,598 - INFO - #Precursors: Shared peptides = 766; Unique peptides = 7826
2025-03-26 22:19:57,629 - INFO - Calculating protein group-level FDRs
2025-03-26 22:19:57,632 - INFO - #Protein groups: Target = 802; Decoys = 436
2025-03-26 22:19:57,633 - INFO -     Entrapments = 429; Targets-Entrapments = 373
2025-03-26 22:19:57,633 - INFO - #Target protein groups at 1% decoy FDR: 369
2025-03-26 22:19:57,634 - INFO - #Target protein groups at 1% entrapment FDR: 58
2025-03-26 22:19:57,634 - INFO - Decoy FDR at 1% entrapment FDR: 0.0027
2025-03-26 22:19:57,635 - INFO - Entrapment FDR at 1% decoy FDR: 0.027
2025-03-26 22:19:57,685 - INFO - Rescuing threshold: protein score = 2.72, peptide PEP = 0.00193
2025-03-26 22:19:57,687 - INFO - Redoing protein grouping u

2025-03-26 22:19:57,719 - INFO - #Precursors: Shared peptides = 711; Unique peptides = 7881
2025-03-26 22:19:57,759 - INFO - Calculating protein group-level FDRs
2025-03-26 22:19:57,762 - INFO - #Protein groups: Target = 805; Decoys = 438
2025-03-26 22:19:57,763 - INFO -     Entrapments = 429; Targets-Entrapments = 376
2025-03-26 22:19:57,763 - INFO - #Target protein groups at 1% decoy FDR: 371
2025-03-26 22:19:57,764 - INFO - #Target protein groups at 1% entrapment FDR: 0
2025-03-26 22:19:57,764 - INFO - Decoy FDR at 1% entrapment FDR: 0.0027
2025-03-26 22:19:57,765 - INFO - Entrapment FDR at 1% decoy FDR: 0.027


6. Turn the results into a dataframe and remove unused columns

In [8]:
protein_groups_df = pd.DataFrame(results.protein_group_results)
protein_groups_df = protein_groups_df.drop(columns=['precursorQuants', 'extraColumns'])

7. Filter at your desired protein group-level FDR.

In [9]:
protein_groups_df = protein_groups_df.loc[protein_groups_df['qValue'] < 0.01]
protein_groups_df

,proteinIds,majorityProteinIds,peptideCountsUnique,bestPeptide,numberOfProteins,qValue,score,reverse,potentialContaminant
0,HPRR3950217_poolB,HPRR3950217_poolB,5,GAYAGITGNPSGWHSSSR,1,0.002681,3.304491,,
1,HPRR2760146_poolA,HPRR2760146_poolA,6,ELLEEMHGEREEGCTQENTESEYYAK,1,0.002681,3.304491,,
2,HPRR3450023_poolB,HPRR3450023_poolB,1,CSDNYEAQAEK,1,0.002681,3.304491,,
3,HPRR4290343_poolB,HPRR4290343_poolB,2,VYYFNHITNASQWERPSGNSSSGGK,1,0.002681,3.304491,,
4,HPRR3000481_poolA,HPRR3000481_poolA,3,ASAQQENSSTCIGSAIK,1,0.002681,3.304491,,
...,...,...,...,...,...,...,...,...,...
368,HPRR4350010_poolB,HPRR4350010_poolB,1,YHNLSLYTE,1,0.002681,3.011993,,
369,HPRR4430109_poolB,HPRR4430109_poolB,1,FLRPWYF,1,0.002681,3.011993,,
370,rev_HPRR4320363_entrapment,rev_HPRR4320363_entrapment,2,LGEAVNTEPEKESHKWR,1,0.005348,2.836324,+,
371,HPRR4320515_entrapment,HPRR4320515_entrapment,1,LDTGGDLSRGEVPEPLCACR,1,0.005348,2.715377,,
